<h1> CIFAR10 classification using CNNs</h1>

In this Project we attempt to implement a CNN classfier for STL dataset using Tensorflow Estimators. The model code is defined in a way that it can be changed easily. The goal of the next cell is to copy the dataset from Google Drive to the remote Disk and do the computations. 

<h3> Dataset evaluation</h3>
This cell simply tries to check if the importing process of dataset has worked well and outputs the shape of one of the samples in dataset. So run this if you just wanna check the importing process otherwise do not tend to run this cause all the dataset will be loaded in RAM, thus in the next cells as we do have some other imports, you may get an memory error.

In [2]:
from matplotlib import pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import time
import sklearn
from google.colab import drive

drive.mount('content/')

img_size = 32
img_flat_size = 32 * 32
learning_rate = 0.0001
batch_size = 64
local_direc = "./model"
drive_direc = "./content/My Drive/Projects/Colab/NN/HW4/part5/model"


Drive already mounted at content/; to attempt to forcibly remount, call drive.mount("content/", force_remount=True).


<h3>Model and input functions definition</h3>
In the next cell a dynamic model of the CNNs has been implemented and the dataset will be imported as well. 

In [3]:
def eval_conf_matrix(labels, predictions, params):

    num_classes = params['num_classes']
    conf_matrix = tf.confusion_matrix(labels, predictions, num_classes=num_classes)
    conf_matrix_sum = tf.Variable(tf.zeros(shape = (num_classes, num_classes), dtype=tf.int32),
                                  name="confusion_matrix",
                                  trainable=False,
                                  collections=[tf.GraphKeys.LOCAL_VARIABLES])

    update_op = tf.assign_add(conf_matrix_sum, conf_matrix)
    return tf.convert_to_tensor(conf_matrix_sum), update_op

  
  # This is the part where we definr the model
def model_fn(features, labels, mode, params):
    
  global_step = tf.train.get_global_step()
    
  # The input layer
  input_layer = tf.reshape(features, [-1, img_size, img_size, 3])
  
  ##################################################################
  # This part is the implementation of the first layer of
  #   convolution and subsampling.
  ##################################################################
  # Convolution layer
  conv = tf.layers.conv2d(
        inputs=input_layer,
        filters=params['conv_layers'][0]['filters'],
        kernel_size=params['conv_layers'][0]['kernel_size'], 
        padding="same",
        activation=tf.nn.relu)


  # Pooling layer
  pool = tf.layers.max_pooling2d(
        inputs=conv, 
        pool_size=params['conv_layers'][0]['pooling_size'], 
        strides=params['conv_layers'][0]['strides'])

  ##################################################################
  # This part is for dynamicness of the model. The structure of the
  #  model will be imported in params parameter and using that 
  #  the NN model will be implemented, thus there is no need to change
  #  the model_fn function if you want to change the structure.
  ##################################################################  
  for conv_pool_layer in params['conv_layers'][1:]:
    
    # Interior Convolutional layer 
    conv = tf.layers.conv2d(
        inputs=pool,
        filters=conv_pool_layer['filters'],
        kernel_size=conv_pool_layer['kernel_size'], 
        padding="same",
        activation=tf.nn.relu)


    # Interior Pooling layer 
    pool = tf.layers.max_pooling2d(
        inputs=conv, 
        pool_size=conv_pool_layer['pooling_size'], 
        strides=conv_pool_layer['strides'])

  
  ##################################################################
  # In this part we implement the dense layers. These layers are 
  #  dynamic as well, thus there is no need to change this part if 
  #  structure reformation is required. Just conduct the changes
  #  in params parameter passed to this method.
  ##################################################################  
  
  # Size of the output of the last pooling layer to feed to the dense layers
  new_size = int(img_size / (2 ** len(params['conv_layers'])))
  
  # raveled form of the outputs of last pooling layer
  flat_output = tf.reshape(pool, [-1, new_size * new_size * params['conv_layers'][-1]['filters']])

  
  # First dense layer
  dense = tf.layers.dense(inputs=flat_output, 
                          units=params['dense_layers'][0]['units'], 
                          activation=tf.nn.relu)
  
  # dropout applied to the dense layers
  dropout = tf.layers.dropout(inputs=dense, rate=params['dense_layers'][0]['dropout'])

  for dense_layer in params['dense_layers'][1:]:
    
    # interior dense layers
    dense = tf.layers.dense(inputs=dropout, 
                          units=dense_layer['units'], 
                          activation=tf.nn.relu)
    
    
    # If dropout is not necessary, set the rate to zero in params dictionary
    dropout = tf.layers.dropout(inputs=dense, rate=dense_layer['dropout'])

  # The logits will be calculated from the last dropout layer
  logits = tf.layers.dense(inputs=dropout, units=params['num_classes'])
  
  prediction_labels = tf.argmax(logits, 1)
  probabilities = tf.nn.softmax(logits)


  predictions = {
      'prediction_labels':prediction_labels,
      'probabilities': probabilities
  }

  #   This part is for the prediction part of the model
  if mode == tf.estimator.ModeKeys.PREDICT:
      return tf.estimator.EstimatorSpec(
          mode = mode,
          predictions = predictions
      )

  accuracy = tf.metrics.accuracy(labels, prediction_labels)
  confusion_matrix_tuple = eval_conf_matrix(labels, prediction_labels, params)

  cross_entropy = tf.losses.sparse_softmax_cross_entropy(labels, logits)

    # This is the evaluation part of the model
  if mode == tf.estimator.ModeKeys.EVAL:
      tf.summary.scalar("Evaluation_accuracy", accuracy[1])
      return tf.estimator.EstimatorSpec(
          mode=mode,
          loss=cross_entropy,
          eval_metric_ops={'eval_accuracy': accuracy,
                           'confusion_matrix': confusion_matrix_tuple},
          predictions = predictions,
          evaluation_hooks=None
      )

  optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
  train_op = optimizer.minimize(cross_entropy, global_step=global_step)

  train_hook_list = []
  train_tensor_logs = {
      'accuracy':accuracy[1],
      'loss':cross_entropy,
      'global_step':global_step
  }

  train_hook_list.append(tf.train.LoggingTensorHook(
      tensors=train_tensor_logs, every_n_iter=100
  ))

  #   This is the training part of the model
  if mode == tf.estimator.ModeKeys.TRAIN:
      tf.summary.scalar("Training_accuracy", accuracy[1])
      return tf.estimator.EstimatorSpec(
          mode=mode,
          loss=cross_entropy,
          train_op=train_op,
          eval_metric_ops=None,
          training_hooks=train_hook_list
      )


def STL_classifier(_):
  
  (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
  
  x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, random_state=10)
  
  y_train = np.asarray(y_train, dtype=np.int64)
  y_val = np.asarray(y_val, dtype=np.int64)
  y_test = np.asarray(y_test, dtype=np.int64)
  x_train = np.asarray(x_train, dtype=np.float64)
  x_test = np.asarray(x_test, dtype=np.float64)
  x_val = np.asarray(x_val, dtype=np.float64)

  train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x=x_train,
      y=y_train,
      batch_size=batch_size,
      num_epochs=7,
      shuffle=True)

  eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x=x_val,
      y=y_val,
      batch_size=batch_size,
      num_epochs=1,
      shuffle=False)

  test_input_fn = tf.estimator.inputs.numpy_input_fn(
      x=x_test,
      y=y_test,
      batch_size=batch_size,
      num_epochs=1,
      shuffle=False)

  saving_configuration = tf.estimator.RunConfig(save_checkpoints_secs=300, keep_checkpoint_max=2)
  
  image_classifier = tf.estimator.Estimator(
      model_dir=drive_direc,
      model_fn=model_fn,
      config=saving_configuration,
      params={
          'num_classes':10,
          'conv_layers': [{'kernel_size':[5, 5], 'filters':32, 'pooling_size':[2, 2], 'strides':2},
                          {'kernel_size':[5, 5], 'filters':32, 'pooling_size':[2, 2], 'strides':2}],
          'dense_layers': [{'units': 512, 'dropout': 0.4}]
      })

  start = time.time()

  for i in range(0):
      image_classifier.train(input_fn=train_input_fn, steps = None)
      metrices = image_classifier.evaluate(input_fn=eval_input_fn)
      print("\n\n******************************************************\nConfusion matrix for evaluation data:\n\n"
            + str(metrices['confusion_matrix']) + "\n\n\nOther evaluation matrices:"
            + "\nEvaluation accuracy:%.2f\tLoss:%.2f\tGlobal step:%d"%(metrices['eval_accuracy'] * 100, metrices['loss'], metrices['global_step'])
            + "\n******************************************************")

  end = time.time()
  print("******************************************************\nTraining time:%.4s seconds"%str(end - start)
        + "\n******************************************************\n\n")

  predictions = image_classifier.predict(
      input_fn=test_input_fn,
      yield_single_examples=False
  )

  accuracy = 0
  i = 0
  conf_matrix = np.zeros((10, 10))
  for epoch_result in predictions:
      accuracy += sklearn.metrics.accuracy_score(y_test[i * batch_size: (i + 1) * batch_size], 
                                                 epoch_result['prediction_labels'])

      
      conf_matrix += sklearn.metrics.confusion_matrix(y_test[i * batch_size: (i + 1) * batch_size], 
                                                      epoch_result['prediction_labels'], labels=range(10))
      
      i = i + 1
  print(i)
  print("\n******************************************************\nConfusion matrix for test data:\n\n" + str(conf_matrix)
        + "\n******************************************************")

  print(conf_matrix)
  
  print("\n******************************************************\nAccuracy for test data: %.2f"
        "\n******************************************************\n"%(accuracy / i * 100))

if __name__ == '__main__':
  tf.app.run(STL_classifier)

170500096/170498071 [==============================] - 4s 0us/step
INFO:tensorflow:Using config: {'_model_dir': './content/My Drive/Projects/Colab/NN/HW4/part5/model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 300, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 2, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb70dbd8ef0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
******************************************************
Training time:2.38 se

SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

!rm -rd model
!mkdir model
!cp -rd ./content/My\ Drive/Projects/Colab/NN/HW4/part5/model/* ./model/

get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format("./model")
)

get_ipython().system_raw('./ngrok http 6006 &')

! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"